In [1]:
import pandas as pd
import numpy as np
from global_functions import get_newest_date_file

In [2]:
pd.options.display.max_columns = None

In [3]:
df_2020 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\BASE_YEAR\2020_jtmt_forcast_full_240506.xlsx").set_index('Taz_num').sort_index() 
df_jtmt_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_jtmt.xlsx").set_index('Taz_num').sort_index() 
df_iplan_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_iplan.xlsx").set_index('Taz_num').sort_index() 
df_bau_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_bau.xlsx").set_index('Taz_num').sort_index() 

In [4]:
BaseProjections_version = get_newest_date_file(r'C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\JTMT', 'BaseProjections2025_')

years = ['2025', '2030','2035', '2040', '2045', '2050']

senarios = ['JTMT', 'IPLAN', 'BAU']

BaseProjections_jtmt_2020 = pd.read_excel(fr'C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\BASE_YEAR\BaseProjections2020_{BaseProjections_version}.xlsx')

for sen in senarios:
    for year in years:
        globals()[f'BaseProjections_{sen}_{year}'] = pd.read_excel(fr'C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\{sen}\BaseProjections{year}_{BaseProjections_version}_{sen.lower()}.xlsx')
        globals()[f'values_list_{sen}'] = [np.nan]

In [5]:
df_jtmt_2050 = pd.merge(df_jtmt_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')
df_iplan_2050 = pd.merge(df_iplan_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')
df_bau_2050 = pd.merge(df_bau_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')

In [6]:
SCHOOLDISTS = pd.pivot_table(df_jtmt_2050, index=['SCHOOLDIST'], aggfunc='sum')

In [7]:
def convert_to_int(float_list):
  int_list = []
  for num in float_list:
    if not isinstance(num, float):
      raise ValueError("Input list must contain only floats")
    # Check if conversion to int results in a loss of precision
    if num != int(num):
      raise ValueError(f"Float {num} cannot be converted to int without losing precision")
    int_list.append(int(num))
  return int_list

In [8]:
def add_total_by_type_row(output_df, column_input_name, column_input_value, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = df[df[f'{column_input_name}'] == column_input_value] 
        values_list.append(len(val))

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [9]:
def add_total_row(output_df, column_input_name, column_output_name):
    values_list = []

    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        if column_input_name in df.columns:
            val = df[f'{column_input_name}'].sum()
            # val = df.pivot_table(index='main_secto', values=f'{column_input_name}', aggfunc="sum", fill_value=0).astype('float32').sum()
            values_list.append("{:,}".format(int(val)))
        else:
            values_list.append(np.nan)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [10]:
def add_taz_num_is_unique_row(output_df, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = df_2020.index.is_unique
        values_list.append(val)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [11]:
def add_taz_num_count_row(output_df, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = len(df.index)
        values_list.append(val)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [12]:
def add_percentage_growth_row(current, previous, output_df, column_output_name):
    values_list = []

    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        if len(list(df)) == 135:
                values_list.append(np.nan)

        else:
            if df[f'{current}'].sum() == df[f'{previous}'].sum():
                values_list.append(100.0)
            else:
                try:
                    values_list.append((df[f'{current}'].sum() / df[f'{previous}'].sum()) * 100.0)
                except ZeroDivisionError:
                    values_list.append(0)
    
    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [13]:
def add_students_by_SCHOOLDIST(output_df, column_key_name, column_input_name, column_input_value, column_output_name):
    values_list = []
    
    # if column_key_name in df.columns:
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = pd.pivot_table(df, index=[f'{column_input_name}'], aggfunc="sum")
        if column_key_name in val.columns:
            values_list.append(val[column_key_name][column_input_value])
        else:
            values_list.append(np.nan)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [14]:
for sen in senarios:
        globals()[f'pivot_sector_{sen.lower()}'] = pd.pivot_table(locals()[f'df_{sen.lower()}_2050'], index=['main_secto'], aggfunc="sum")

In [15]:
def add_percentage_growth_by_sector_row(current_year, previous_year, output_df):
        values_list = [np.nan]

        for i, index in enumerate(pivot_sector_jtmt.index):
                for sen in [pivot_sector_jtmt, pivot_sector_iplan, pivot_sector_bau]:
                        if previous_year == '2020':
                                if current_year == previous_year:
                                        values_list.append(100)
                                else:
                                        val = ((sen.iloc[i][f'pop_without_dorms_yeshiva_{current_year}'] / sen.iloc[i]['pop_without_dorms_yeshiva']) * 100.0)
                                        if val == float("inf"):
                                                values_list.append(0)
                                        else:
                                                values_list.append(val)
                        else:
                                if current_year == previous_year:
                                        values_list.append(100)
                                else:
                                        val = ((sen.iloc[i][f'pop_without_dorms_yeshiva_{current_year}'] / sen.iloc[i][f'pop_without_dorms_yeshiva_{previous_year}']) * 100.0)
                                        if val == float("inf"):
                                                values_list.append(0)
                                        else:
                                                values_list.append(val)
                output_df.loc[f'percentage growth {pivot_sector_jtmt.index[i]} {previous_year}-{current_year}'] = np.float32(values_list)
                values_list = [np.nan]
                
        return output_df

In [16]:
monitoring_df = pd.DataFrame(columns=['2020', 'JTMT', 'iplan', 'bau'])
monitoring_df_1 = pd.DataFrame(columns=['2020', 'JTMT', 'iplan', 'bau'])

In [17]:
SCHOOLDIST_list = list(df_2020['SCHOOLDIST'].value_counts().index)
sectors_list = list(df_2020['main_secto'].value_counts().index)
zonetypes_list = list(df_2020['zonetype'].value_counts().index)
jeru_metros_list = list(df_2020['jeru_metro'].value_counts().index)
in_jerusals_list = list(df_2020['in_jerusal'].value_counts().index)

In [18]:
# אוכלוסיה
monitoring_df = add_total_row(monitoring_df, 'pop_without_dorms_yeshiva', 'pop_2020')
monitoring_df = add_total_row(monitoring_df, 'pop_without_dorms_yeshiva_2050', 'pop_2050')

# מועסקים
monitoring_df = add_total_row(monitoring_df, 'total_emp', 'emp_2020')
monitoring_df = add_total_row(monitoring_df, 'total_emp_2050', 'emp_2050')

# מספר א"ת לפי סקטור
for item in sectors_list:
    monitoring_df = add_total_by_type_row(monitoring_df, 'main_secto', item, f'Taz_U_{item}')

# סטודנטים
monitoring_df = add_total_row(monitoring_df, 'univ', 'univ_2020')
monitoring_df = add_total_row(monitoring_df, 'univ_2050', 'univ_2050')

# בתי ספר
monitoring_df = add_total_row(monitoring_df, 'student', 'student_2020')
monitoring_df = add_total_row(monitoring_df, 'student_2050', 'student_2050')

# ישיבות
monitoring_df = add_total_row(monitoring_df, 'student_yeshiva_and_kollim', 'student_yeshiva_and_kollim_2020')
monitoring_df = add_total_row(monitoring_df, 'student_yeshiva_and_kollim_2050', 'student_yeshiva_and_kollim_2050')

# zonetype
for item in zonetypes_list:
    monitoring_df = add_total_by_type_row(monitoring_df, 'zonetype', item, f'zonetype_{item}')

# jeru_metro
for item in jeru_metros_list:
    monitoring_df = add_total_by_type_row(monitoring_df, 'jeru_metro', item, f'jeru_metro_{item}')

# in_jerusal
for item in in_jerusals_list:
    monitoring_df = add_total_by_type_row(monitoring_df, 'in_jerusal', item, f'in_jerusal_{item}')

# אין א"ת שחוזר פעמיים
monitoring_df = add_taz_num_is_unique_row(monitoring_df, 'taz_num_is_unique')
# מספר א"ת
monitoring_df = add_taz_num_count_row(monitoring_df, 'taz_num_count')

In [19]:
# גידול אוכלוסיה כל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2025', 'pop_without_dorms_yeshiva', monitoring_df, 'percentage growth pop 2020-2025')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2030', 'pop_without_dorms_yeshiva_2025', monitoring_df, 'percentage growth pop 2025-2030')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2035', 'pop_without_dorms_yeshiva_2030', monitoring_df, 'percentage growth pop 2030-2035')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2040', 'pop_without_dorms_yeshiva_2035', monitoring_df, 'percentage growth pop 2035-2040')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2045', 'pop_without_dorms_yeshiva_2040', monitoring_df, 'percentage growth pop 2040-2045')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2050', 'pop_without_dorms_yeshiva_2045', monitoring_df, 'percentage growth pop 2045-2050')

# גידול סטודנטים כל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('univ_2025', 'univ', monitoring_df, 'percentage growth univ 2020-2025')
monitoring_df = add_percentage_growth_row('univ_2030', 'univ_2025', monitoring_df, 'percentage growth univ 2025-2030')
monitoring_df = add_percentage_growth_row('univ_2035', 'univ_2030', monitoring_df, 'percentage growth univ 2030-2035')
monitoring_df = add_percentage_growth_row('univ_2040', 'univ_2035', monitoring_df, 'percentage growth univ 2035-2040')
monitoring_df = add_percentage_growth_row('univ_2045', 'univ_2040', monitoring_df, 'percentage growth univ 2040-2045')
monitoring_df = add_percentage_growth_row('univ_2050', 'univ_2045', monitoring_df, 'percentage growth univ 2045-2050')

# גידול תלמידי ישיבה כל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('student_yeshiva_and_kollim_2025', 'student_yeshiva_and_kollim', monitoring_df, 'percentage growth student_yeshiva_and_kollim 2020-2025')
monitoring_df = add_percentage_growth_row('student_yeshiva_and_kollim_2030', 'student_yeshiva_and_kollim_2025', monitoring_df, 'percentage growth student_yeshiva_and_kollim 2025-2030')
monitoring_df = add_percentage_growth_row('student_yeshiva_and_kollim_2035', 'student_yeshiva_and_kollim_2030', monitoring_df, 'percentage growth student_yeshiva_and_kollim 2030-2035')
monitoring_df = add_percentage_growth_row('student_yeshiva_and_kollim_2040', 'student_yeshiva_and_kollim_2035', monitoring_df, 'percentage growth student_yeshiva_and_kollim 2035-2040')
monitoring_df = add_percentage_growth_row('student_yeshiva_and_kollim_2045', 'student_yeshiva_and_kollim_2040', monitoring_df, 'percentage growth student_yeshiva_and_kollim 2040-2045')
monitoring_df = add_percentage_growth_row('student_yeshiva_and_kollim_2050', 'student_yeshiva_and_kollim_2045', monitoring_df, 'percentage growth student_yeshiva_and_kollim 2045-2050')

# גידול אוכלוסיה כל חומש בכל תרחיש לפי מגזר
monitoring_df = add_percentage_growth_by_sector_row('2025', '2020', monitoring_df)
monitoring_df = add_percentage_growth_by_sector_row('2030', '2025', monitoring_df)
monitoring_df = add_percentage_growth_by_sector_row('2035', '2030', monitoring_df)
monitoring_df = add_percentage_growth_by_sector_row('2040', '2035', monitoring_df)
monitoring_df = add_percentage_growth_by_sector_row('2045', '2040', monitoring_df)
monitoring_df = add_percentage_growth_by_sector_row('2050', '2045', monitoring_df)

C:\Users\dpere\AppData\Local\Temp\ipykernel_33272\1674840370.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  val = ((sen.iloc[i][f'pop_without_dorms_yeshiva_{current_year}'] / sen.iloc[i]['pop_without_dorms_yeshiva']) * 100.0)


In [20]:
# # SCHOOLDIST 2020
# for item in SCHOOLDIST_list:
#     monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2020')

# # SCHOOLDIST 2025
# for item in SCHOOLDIST_list:
#     monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2025', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2025')

# # SCHOOLDIST 2030
# for item in SCHOOLDIST_list:
#     monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2030', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2030')

# # SCHOOLDIST 2035
# for item in SCHOOLDIST_list:
#     monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2035', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2035')

# # SCHOOLDIST 2040
# for item in SCHOOLDIST_list:
#     monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2040', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2040')

# # SCHOOLDIST 2045
# for item in SCHOOLDIST_list:
#     monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2045', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2045')

# # SCHOOLDIST 2050
# for item in SCHOOLDIST_list:
#     monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2050', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2050')

In [21]:
years = ['2020', '2025', '2030','2035', '2040', '2045', '2050']
values_list = [np.nan]

In [22]:
for previous, current in zip(years, years[1:]):
    for sen in senarios:
        BaseProjections_current_sector = pd.pivot_table(globals()[f'BaseProjections_{sen}_{current}'], index=['sector'], aggfunc="sum")
        if previous == '2020':
            BaseProjections_previous_sector = pd.pivot_table(BaseProjections_jtmt_2020, index=['sector'], aggfunc="sum")
        else:
            BaseProjections_previous_sector = pd.pivot_table(globals()[f'BaseProjections_{sen}_{previous}'], index=['sector'], aggfunc="sum")

        previous_total = BaseProjections_previous_sector.iloc[3]['age15_19'] + BaseProjections_previous_sector.iloc[3]['age20_24']
        BaseProjections_age15_24_U_previous_total = previous_total / 2

        current_total = BaseProjections_current_sector.iloc[3]['age15_19'] + BaseProjections_current_sector.iloc[3]['age20_24']
        BaseProjections_age15_24_U_current_total = current_total / 2

        val = ((BaseProjections_age15_24_U_current_total / BaseProjections_age15_24_U_previous_total) * 100.0)
        values_list.append(val)
    monitoring_df.loc[f'percentage growth U_Orthodox age15_24 {previous}-{current}'] = values_list
    values_list = [np.nan]

In [23]:
for i, sector in enumerate(pivot_sector_jtmt.index):
    for previous, current in zip(years, years[1:]):
        for sen in senarios:
            BaseProjections_current_sector = pd.pivot_table(globals()[f'BaseProjections_{sen}_{current}'], index=['sector'], aggfunc="sum")
            if previous == '2020':
                BaseProjections_previous_sector = pd.pivot_table(BaseProjections_jtmt_2020, index=['sector'], aggfunc="sum")
            else:
                BaseProjections_previous_sector = pd.pivot_table(globals()[f'BaseProjections_{sen}_{previous}'], index=['sector'], aggfunc="sum")

            previous_total = BaseProjections_previous_sector.iloc[i]['age20_24']/ 5 * 3 + BaseProjections_previous_sector.iloc[i]['age25_29']/ 5 * 4
            current_total = BaseProjections_current_sector.iloc[i]['age20_24']/ 5 * 3 + BaseProjections_current_sector.iloc[i]['age25_29']/ 5 * 4

            val = ((current_total / previous_total) * 100.0)
            values_list.append(val)
        monitoring_df.loc[f'percentage growth {sector} age20_29 {previous}-{current}'] = values_list
        values_list = [np.nan]

In [24]:
# BaseProjections_senarios = ['2020' ,'JTMT', 'IPLAN', 'BAU']

age6_18_list = [np.nan]
students_list = [np.nan]
different_list = [np.nan]

for i, SCHOOLDIST in enumerate(SCHOOLDISTS.index):
    for year in years:
        
        for sen in senarios:
            if year == '2020':
                globals()[f'BaseProjections_students_{sen}_2020'] = pd.merge(BaseProjections_jtmt_2020, df_2020[['SCHOOLDIST']],left_on='TAZ', right_on='Taz_num')
                globals()[f'BaseProjections_students_{sen}_2020'] = pd.pivot_table(globals()[f'BaseProjections_students_{sen}_2020'], index=['SCHOOLDIST'], aggfunc="sum")
            else:
                globals()[f'BaseProjections_students_{sen}_{year}'] = pd.merge(globals()[f'BaseProjections_{sen}_{year}'],df_2020[['SCHOOLDIST']],left_on='TAZ', right_on='Taz_num')
                globals()[f'BaseProjections_students_{sen}_{year}'] = pd.pivot_table(globals()[f'BaseProjections_students_{sen}_{year}'], index=['SCHOOLDIST'], aggfunc='sum')
            
            age6_9 = globals()[f'BaseProjections_students_{sen}_{year}']['age5_9']/5*4
            age15_18 = globals()[f'BaseProjections_students_{sen}_{year}']['age15_19']/5*4
            age6_18 = age6_9 + globals()[f'BaseProjections_students_{sen}_{year}']['age10_14'] + age15_18

            different_list.append(age6_18[i] - globals()[f'BaseProjections_students_{sen}_{year}']['student'][i])
            age6_18_list.append(age6_18[i])
            students_list.append(globals()[f'BaseProjections_students_{sen}_{year}']['student'][i])

        monitoring_df.loc[f'students at district {i} {year}'] = students_list
        monitoring_df.loc[f'age6_18 at district {i} {year}'] = age6_18_list
        monitoring_df.loc[f'different at district {i} {year}'] = different_list

        age6_18_list = [np.nan]
        students_list = [np.nan]
        different_list = [np.nan]

In [25]:
# monitoring_df = monitoring_df.fillna('NaN')

In [26]:
def highlight_cols(s): 
    if s > 100:
        color ='red'
    elif s < -100:
        color = 'red'
    else: 
        color = 'none'
    return 'background-color: % s' % color

idx = pd.IndexSlice[monitoring_df.index.get_level_values(0).str.contains('different'), :]
monitoring_df = monitoring_df.style.applymap(highlight_cols, subset=idx)

# def highlight_cols(s): 
#     color = 'ed' if int(s) > 100 else ''
#     return 'background-color: %s' % color

# idx = pd.IndexSlice[monitoring_df.index.get_level_values(0).str.contains('different'), :]
# monitoring_df.style.applymap(highlight_cols, subset=idx)

C:\Users\dpere\AppData\Local\Temp\ipykernel_33272\4160988309.py:11: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  monitoring_df = monitoring_df.style.applymap(highlight_cols, subset=idx)


In [27]:
monitoring_df.to_excel('monitoring_df_fixed_3.xlsx')
# monitoring_df
monitoring_df

,2020,JTMT,iplan,bau
pop_2020,"1,758,461","1,758,461","1,758,461","1,758,461"
pop_2050,nan,"7,555,735","7,278,700","7,759,649"
emp_2020,"482,716","482,716","482,716","482,716"
emp_2050,nan,"989,702","837,076","962,301"
Taz_U_Jewish,482,482,472,450
Taz_U_U_Orthodox,174,174,184,206
Taz_U_Palestinian,157,157,157,157
Taz_U_Arab,86,86,86,86
Taz_U_arabs_behined_seperation_wall,5,5,5,5
univ_2020,"81,184","81,184","81,184","81,184"


In [28]:
-522.200000 < -100

True